In [106]:

using System;
using System.Linq;
using System.Threading;
using System.Threading.Tasks;
//
// Вычисление определенного интеграла
//
class DefiniteIntegral
{
    //
    // a, b - границы отрезка, на котором происходит вычисление опредленного интеграла
    // function - функция, для которой вычисляется определнный интеграл
    // step - размер одного шага разбиения
    // threadsNumber - число потоков, которые используются для вычислений
    //
    public static double Solve(double a, double b, Func<double, double> function, double step, int threadsNumber)
    {
       
        
        double thread_Step = (b - a) / threadsNumber;

       
        Task[] tasks = new Task[threadsNumber];

        
        long result = 0;

       
        Barrier barrier = new Barrier(threadsNumber);

        
        for (int i = 0; i < threadsNumber; i++)
        {
            int thread_Index = i;
            tasks[i] = Task.Run(() =>
            {
                
                double local_Result = 0;
                double local_Step = step;
                if (thread_Index == threadsNumber - 1)
                    local_Step = (b - a) - thread_Index * thread_Step;
                for (double x = a + thread_Index * thread_Step; x < a + (thread_Index + 1) * thread_Step; x += local_Step)
                    local_Result += (function(x) + function(x + local_Step)) * local_Step / 2;

                
                Interlocked.Add(ref result, (long)(local_Result * 1e15));

                
                barrier.SignalAndWait();
            });
        }

        
        Task.WaitAll(tasks);

      
        return Math.Round(result / 1e15);
    }
}




In [107]:
#!csharp

#r "nuget: xunit, 2.8.1"

using Xunit;

var X = (double x) => x;
var SIN = (double x) => Math.Sin(x);

Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, X, 1e-4, 2), 1e-4);

Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, SIN, 1e-5, 8), 1e-4);

Assert.Equal(50, DefiniteIntegral.Solve(0, 10, X, 1e-6, 8), 1e-5);

Installed Packages xunit, 2.8.1